In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import os
# import cudf as cd
# import cupy as cp
# import dask_cudf as dc
# from cuml.decomposition import pca
import numpy as np

In [ ]:
# path = '/mnt/h/work_projects/College/DM_Projects/course_project_2/data/archive 1'
path = 'H:/work_projects/College/DM_Projects/course_project_2/data/archive 1'
dir_list = os.listdir(path)
df = pd.DataFrame()
for i in dir_list:
    if df.empty:
        df = pd.read_csv(path + '/' + i, sep='|', na_values='-')
    else:
        df = pd.concat([pd.read_csv(path + '/' + i, sep='|', na_values='-'), df], axis=0)

In [3]:
df = df.drop(columns=['uid', 'local_orig', 'local_resp', 'tunnel_parents', 'id.orig_h', 'id.resp_h'])

In [4]:
df['final_label'] = df['label'].astype('string') + '-' + df['detailed-label'].fillna('').astype('string')

In [5]:
df = df.drop(['label', 'detailed-label'], axis=1)

In [6]:
df = pd.get_dummies(df, columns=['proto', 'conn_state', 'service'])
df['history'] = pd.factorize(df['history'])[0] + 1
df['final_label'] = pd.factorize(df['final_label'])[0]

In [7]:
df['duration'] = df['duration'].astype('float')
df['orig_bytes'] = df['orig_bytes'].astype('float')
df['resp_bytes'] = df['resp_bytes'].astype('float')

In [8]:
df.head()

,ts,id.orig_p,id.resp_p,duration,orig_bytes,resp_bytes,missed_bytes,history,orig_pkts,orig_ip_bytes,...,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,service_dhcp,service_dns,service_http,service_irc,service_ssh,service_ssl
0,1.532510e+09,40008.0,123.0,0.004751,48.0,48.0,0.0,1,1.0,76.0,...,False,True,False,False,False,False,False,False,False,False
1,1.532511e+09,57849.0,22.0,108.421563,2376.0,96.0,0.0,2,5.0,332.0,...,False,False,False,False,False,False,False,False,False,False
2,1.532512e+09,59670.0,22.0,0.001244,21.0,0.0,0.0,3,3.0,189.0,...,False,False,False,False,False,False,False,False,False,False
3,1.532512e+09,59701.0,22.0,0.000738,21.0,0.0,0.0,3,3.0,189.0,...,False,False,False,False,False,False,False,False,False,False
4,1.532512e+09,36199.0,53.0,NaN,NaN,NaN,0.0,4,1.0,58.0,...,False,False,False,False,False,True,False,False,False,False


In [9]:
df.shape

(25000363, 35)

In [10]:
x = df.drop(columns=['final_label'])
y = df['final_label']
del df

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
del x
del y

In [ ]:
model = XGBClassifier(tree_method="hist", device="cuda", metric='multi:softmax', num_class=13)
model.fit(x_train, y_train)
predictions = model.predict(x_test)

In [17]:
del x_train
del y_train

In [24]:
y_test.value_counts()

final_label
0     2193252
1     1760486
3     1444883
4      847105
9        1639
11       1502
5         727
2         250
6         226
12         12
10          5
8           3
7           1
Name: count, dtype: int64

In [ ]:
predictions = model.predict(x_test)

In [20]:
from sklearn.metrics import accuracy_score
print(accuracy_score(predictions, y_test))

0.9998544021199052


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(predictions, y_test))

0.9998544021199052


In [21]:
from sklearn.metrics import precision_score
precision_score(predictions, y_test.to_numpy(), average=None)

array([0.9997547 , 0.99999489, 0.784     , 1.        , 0.99999882,
       0.8954608 , 0.        , 1.        , 1.        , 0.99938987,
       0.4       , 0.99866844, 1.        ])

In [22]:
from sklearn.metrics import f1_score
f1_score(predictions, y_test.to_numpy(), average=None)

array([0.99987369, 0.99984495, 0.75095785, 0.99999965, 0.99986544,
       0.91048951, 0.        , 1.        , 1.        , 0.99938987,
       0.57142857, 0.99933378, 1.        ])

In [23]:
from sklearn.metrics import recall_score
recall_score(predictions, y_test.to_numpy(), average=None)

c:\Users\prath\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([0.9999927 , 0.99969506, 0.72058824, 0.99999931, 0.9997321 ,
       0.92603129, 0.        , 1.        , 1.        , 0.99938987,
       1.        , 1.        , 1.        ])